In [1]:
import glob
import os
import subprocess
import numpy as np
import time
import datetime

%load_ext autoreload
%autoreload 1
%aimport bb_behavior
%aimport bb_behavior.plot
%aimport bb_behavior.tracking
%aimport bb_behavior.tracking.pipeline

import bb_behavior
import bb_behavior.plot
import bb_behavior.tracking
import bb_behavior.tracking.pipeline

from bb_behavior.tracking.pipeline import get_default_pipeline
from tqdm import tqdm_notebook # progress bar
import math
from bb_tracking.data.constants import DETKEY
#from bb_tracking.tracking import score_id_sim_v
from bb_tracking.tracking import distance_orientations_v, distance_positions_v

from bb_behavior.tracking.pipeline import detect_markers_in_video
from bb_behavior.tracking.pipeline import track_detections_dataframe
from bb_behavior.tracking.pipeline import display_tracking_results

Using Theano backend.


In [62]:
# hyperparameters

tag_pixel_diameter = 50
n_frames = None
fps = 10.0
progress = "tqdm_notebook"
tracker = "../tracker.det_score_fun.frag_score_fun.dill"
confidence_filter_detections = 0.0
confidence_filter_tracks = 0.2
coordinate_scale = 1.0
localizer_threshold="0.65"


default_pipeline = get_default_pipeline(localizer_threshold=localizer_threshold)

df_list = []

for path in glob.glob(os.path.join('../videos/videos_tags/', '*.h264')):  


    frame_info, detections = detect_markers_in_video(path,
                                            pipeline=default_pipeline,
                                            tag_pixel_diameter=tag_pixel_diameter,
                                            n_frames=n_frames,
                                            fps=fps,
                                            progress=progress
                                               )
    tracks = track_detections_dataframe(detections,
                                        tracker=tracker,
                                        confidence_filter_detections=confidence_filter_detections,
                                        confidence_filter_tracks=confidence_filter_tracks,
                                        coordinate_scale=coordinate_scale)
      
    # get date from filename
    date_string = os.path.split(path)[1].split('.')[0].split('_')[1]
    timestamp = time.mktime(datetime.datetime.strptime(date_string, "%Y-%m-%d-%H-%M-%S").timetuple())
    
    
    #  transform tracks df to : bee_id, [list of positions (x,y)], [list of timestamps], timestamp_of video
    grouped = tracks.groupby('track_id',  as_index=False)['xpos','ypos','timestamp']
    df = grouped.aggregate(lambda x: list(x))
    df = df.drop('track_id', 1)
    
    df['timestamp_video'] = timestamp
    df = df.rename(columns={'timestamp': 'timestamps'})

    df_list.append(df)
    # ma
    
    break
    
# TODO: bind dataframes together

/home/tscho/.local/lib/python3.6/site-packages/keras/backend/theano_backend.py:1032: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode='max')
/home/tscho/.local/lib/python3.6/site-packages/keras/backend/theano_backend.py:1032: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode='max')
/home/tscho/.local/lib/python3.6/site-packages/keras/backend/theano_backend.py:1032: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode='max')
Widget Javascript not detected.  It may not be installed or enabled properly.


/home/tscho/anaconda3/lib/python3.6/site-packages/sklearn/base.py:312: UserWarning: Trying to unpickle estimator StandardScaler from version 0.20.0 when using version 0.19.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/tscho/anaconda3/lib/python3.6/site-packages/sklearn/base.py:312: UserWarning: Trying to unpickle estimator LinearSVC from version 0.20.0 when using version 0.19.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/tscho/anaconda3/lib/python3.6/site-packages/sklearn/base.py:312: UserWarning: Trying to unpickle estimator Pipeline from version 0.20.0 when using version 0.19.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/tscho/anaconda3/lib/python3.6/site-packages/sklearn/base.py:312: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.20.0 when using version 0.19.0. This might lead to breaking code 

In [65]:
df

,xpos,ypos,timestamps,timestamp_video
0,"[912.4305286407471, 974.5363636016846, 1087.55...","[126.9986743927002, 126.95895576477051, 127.15...","[0.0, 0.1, 0.3, 0.7, 1.5, 2.3, 6.8, 7.5]",1.534634e+09
1,"[549.8731517791748, 512.5459098815918, 462.318...","[186.49464416503906, 173.8266544342041, 123.78...","[3.8, 3.9, 4.0, 4.1, 4.2, 4.3, 4.8, 5.2, 5.3, ...",1.534634e+09
2,"[500.4758186340332, 499.3327169418335, 512.776...","[149.63544845581055, 201.98738861083984, 213.8...","[8.3, 10.1, 10.2, 10.3, 10.4, 10.5, 10.7, 10.8...",1.534634e+09
3,"[1337.3061771392822, 1374.0617427825928, 1362....","[264.9521064758301, 152.6708221435547, 139.647...","[11.9, 12.1, 12.2, 12.3, 12.4, 13.9, 14.0, 14....",1.534634e+09
4,"[1324.5931148529053, 1361.885241508484, 1387.1...","[202.4510498046875, 102.17984199523926, 140.10...","[19.6, 19.8, 20.0]",1.534634e+09
